In [1]:
import numpy as np
import copy
from scipy.sparse import csr_matrix

# Do not try to check it or use, while I will  not correct all steps

# Проверить арифметику для sparse матриц, которые проекции, потому что изначально я писал их для денс, переделать этот момент

In [2]:
# Пока сделал так, что A уже как бы проекция. Соответственно, Omega - tuple (row, col) ее ненулевых елементов

class Problem:
    
    def __init__(self, A, tau, rank):
        """
        Inicialization method
        
        Args:
            A (np.array): given matrix
            tau (float): tolerance, if ||grad||<=tau - early out
            rank (int): Rank of matrix approximate
        
        """
        self.tau = tau
        self.rank = rank
        
        self.A = A
        self.m = A.shape[0]
        self.n = A.shape[1]
        
        # (row, col) of nonzero elements in A.
        self.Omega = A.nonzero()
        
    def projection(Y1, Y2):
        """
        Calculate projection of given matrix Z = Y1Y2.T.
        
        Args:
            Y1 (np.array): First matrix
            Y2 (np.array): Second matrix
        
        Returns:
            sparse csr_matrix - projection on the Omega
        """
        rows = self.Omega[0]
        cols = self.Omega[1]
        
        nnz = rows.shape[0]
        data = np.zeros_like(rows)
        
        for it in range(0, nnz):
            data[it] = np.dot(Y1[rows[it]].T, Y2[col[it]])
        
        proj = csr_matrix((data, (rows, cols)), shape=self.A.shape)
        
        return proj
    
    def grad(self, svd, R):
        """
        Compute gradient in the current point
        
        Args:
            svd (list): SVD of the current point
            R (np.array): Xw - Aw
        Returns:
            np.array, list: Gradient and corresponding matrices [value, [M, Up, Vp]]
        """
        U = svd[0]
        V = svd[2]
        
        Ru = np.dot(R.T, U)
        Rv = np.dot(R, V)
        
        M = np.dot(U.T, Rv)
        
        Up = Rv - np.dot(U, M)
        Vp = Ru - np.dot(V, M.T)
        
        
        grad_value = U.dot(M.dot(V.T)) + Up.dot(V.T) + U.dot(Vp.T)
        grad_list = [M, Up, Vp]
        grad = [grad_value, grad_list]
        
        return grad
    
    def vector_transport(self, X_old_svd, X_new_svd, v_list):
        """
        Calculate transport vector from previous tangent space to new tangent space
        
        Args:
            X_old_svd (list): SVD in the previous point in the form [U, S, V], X = USV.T
            X_new_svd (list): SVD in the new point in the form [U+, S+, V+], X+ = U+S+V+.T
            v_list (list): vector, we need to transport in the form [M, Up, Vp]

        Returns:
            np.array, list: Transport vector and corresponding matrices [value, [M+, Up+, Vp+]]
        """
        
        U_old = X_old_svd[0]
        S_old = X_old_svd[1]
        V_old = X_old_svd[2]
    
        U_plus = X_new_svd[0]
        S_plus = X_new_svd[1]
        V_plus = X_new_svd[2]
        
        M = v_list[0]
        Up = v_list[1]
        Vp = v_list[2]
        
        Av = np.dot(V.T, V_plus)
        Au = np.dot(U.T, U_plus)
        
        B_v = np.dot(Vp.T, V_plus)
        B_u = np.dot(Up.T, U_plus)
        
        M_plus_one = np.dot(Au.T, np.dot(M, Av))
        U_plus_one = np.dot(U, np.dot(M, Av))
        V_plus_one = np.dot(V, np.dot(M.T, Au))
        
        M_plus_two = np.dot(Bu.T, Av)
        U_plus_two = np.dot(Up, Av)
        V_plus_two = np.dot(V, Bu)
        
        M_plus_three = np.dot(Au.T, Bv)
        U_plus_three = np.dot(U, Bv)
        V_plus_three = np.dot(Vp, Au)
        
        M_plus = M_plus_one + M_plus_two + M_plus_three
        Up_plus = U_plus_one + U_plus_two + U_plus_three
        Up_plus = Up_plus - np.dot(U_plus, np.dot(U_plus.T, Up_plus))
        
        Vp_plus = V_plus_one + V_plus_two + V_plus_three
        Vp_plus = Vp_plus - np.dot(V_plus, np.dot(V_plus.T, Vp_plus))
        
        transport_value = np.dot(U_plus, np.dot(M_plus, V_plus.T)) + np.dot(Up_plus, V_plus.T) + np.dot(U_plus, Vp_plus.T)
        transport_list = [M_plus, Up_plus, Vp_plus]
        transport = [transport_value, transport_list]
        return transport
    
    def conjugate_direction(self, X_old_svd, X_new_svd, xi_old, xi_new, eta_old):
        """
        Compute the conjugate direction by PR+ in the new point
        
        Args:
            X_old_svd (list): SVD in the previous point in the form [U, S, V], X = USV.T
            X_new_svd (list): SVD in the new point in the form [U+, S+, V+], X+ = U+S+V+.T
            xi_old (list): tangent vector in the previous point in the form [value, [M, Up, Vp]]
            xi_new (list): tangent vector in the new point in the form [value, [M, Up, Vp]]
            eta_old (list): conjugate vector in the previous point in the form [value, [M, Up, Vp]]

        Returns:
            np.array, list: conjugate vector and corresponding list in the new point [M+, Up+, Vp+]
        """
        # Transport previous gradient and direction to current tangent space:
        xi_bar = self.vector_transport(X_old_svd, X_new_svd, xi_old[1])
        eta_bar = self.vector_transport(X_old_svd, X_new_svd, eta_old[1])
        
        # Compute conjugate direction
        delta = xi_new[0] - xi_bar[0]
        top = np.dot(delta.T, xi_new[0])
        bottom = np.dot(xi_old[0].T, xi_old[0])
        betta = np.max(0, top/bottom)
        eta_value = -xi_new[0] + betta*eta_bar[0]
        
        # Renew eta_list
        M_eta_bar = eta_bar[1][0]
        Up_eta_bar = eta_bar[1][1]
        Vp_eta_bar = eta_bar[1][2]
        
        M_xi = xi_new[1][0]
        Up_xi = xi_new[1][1]
        Vp_xi = xi_new[1][2]
        
        M_eta = -M_xi + betta*M_eta_bar
        Up_eta = -Up_xi + betta*Up_eta_bar
        Vp_eta = -Vp_xi + betta*Vp_eta_bar
        
        eta_list = [M_eta, Up_eta, Vp_eta]
        
        eta = [eta_value, eta_list]
        
        # Compute angle between conjugate direction and gradient:
        
        top = np.dot(eta[0].T, xi_new[0])
        bottom = np.sqrt(np.dot(eta[0].T, eta[0])*np.dot(xi_new[0].T, xi_new[0]))
        alpha = top/bottom
        
        # Reset to gradient if desired:
        if alpha <= 0.1:
            eta_value = xi[0].copy()
            eta_list = xi_new[1].copy()
            eta = [eta_value, eta_list]
        
        return eta
    
    def compute_initial_guess(self, X_new_svd, Rw, eta_new):
        """
        Compute the initial guess for line search t* = argmin_t f(X+t*eta)
        Args:
            X_new_svd (list): SVD in the new point in the form [U, S, V], X = USV.T
            Rw (np.array): Rw = Xw - Aw
            eta_new (list): tangent vector in the new point in the form [value, [M, Up, Vp]]
        Returns:
            t (float)
        
        """
        U = X_new_svd[0]
        V = X_new_svd[2]
        M = eta_new[1][0]
        Up = eta_new[1][1]
        Vp = eta_new[1][2]
        
        Y1 = np.dot(U, M) + Up
        Y1 = np.hstack((Y1, U))
        Y2 = np.hstack((V, Vp))
        
        N = self.projection(Y1, Y2)
        top = np.dot(N.T, Rw)
        bottom = np.dot(N.T, N)
        t = np.trace(top)/ np.trace(bottom)
        return t
            
    
    def compute_retraction(self, X_new_svd, xi_new):
        """
        Compute the retraction by metric projection
        Args:
            X_new_svd (list): SVD in the new point in the form [U, S, V], X = USV.T
            xi_new (list): tangent vector in the new point in the form [value, [M, Up, Vp]]
        Returns:
            retraction (list): Retraction on Mk manifold from tangent space after some step
                                    in the form [U, S, V], and  X = USV.T
        """
        
        k = self.rank
        M = xi_new[1][0]
        Up = xi_new[1][1]
        Vp = xi_new[1][2]
        
        Zero = np.zeros(Up.shape)
        First = np.hstack((Up, Zero))
        (Qu, Ru) = np.linalg.qr(First)
        
        Second = np.hstack((Vp, Zero))
        (Qv, Rv) = np.linalg.qr(Second)
        
        top = np.hstack((S + M, Rv.T))
        bottom = np.hstack((Ru, Zero))
        
        S = np.vstack((top, bottom))
        Us, Ss, Vs = np.linalg.svd(S)
        
        e_mach = np.random.uniform(0, 1, k)
        S_plus = Ss[1:k, 1:k] + np.diag(e_mach)
        
        Current = np.hstack((U, Qu))
        U_plus = np.dot(Current, Us)[:, 1:k]

        Current = np.hstack((V, Qv))
        U_plus = np.dot(Current, Vs)[:, 1:k]
        
        #ans = np.dot(U_plus, np.dot(Sigma_plus, V_plus.T))
        retraction = [U_plus, Sigma_plus, V_plus]
        return retraction
        
    
    def LRGeomCG(self):
        
        # Check Initialization!!!!!
        m = self.m
        n = self.n
        k = self.rank
        
        diag = np.random.uniform(0, 1, k)
        diag = np.sort(diag)[::-1]
        
        S0 = np.diag(diag)
        U0 = np.random.uniform(0, 1, (m, k))
        V0 = np.random.uniforn(0, 1, (n, k))
        
        Qu0, Ru0 = np.linalg.qr(U0)
        Qv0, Rv0 = np.linalg.qr(V0)
        
        X_old_svd = [Qu0, S0, Qv0]
        X_new_svd = copy.deepcopy(X_old_svd)
        
        xi_old_value = np.zeros((m, n))
        xi_old_list = [np.zeros((k, k)), np.zeros((m, k)), np.zeros((n, k))]
        xi_old = [xi_old_value, xi_old_list]
        
        eta_old_value = xi_old.copy()
        eta_old_list = xi_old_list.copy()
        eta_old = [eta_old_value, eta_old_list]
        
        
        while True:
            
            # Projection of X
            Y1 = np.dot(X_new_svd[0], X_new_svd[1])
            Y2 = X_new_svd[2]
            Xw = self.projection(Y1, Y2)
            
            # Calculation P_w(X-A) = Xw-Aw = Xw - A
            Rw = Xw - self.A
            
            # Compute the gradient
            xi_new = self.grad(X_new_svd, Rw)
            
            # Check convergence
            if np.linalg.norm(xi_new[0]) <= self.tau:
                break
            
            # Compute a conjugate direction by PR+
            eta_new = conjugate_direction(self, X_old_svd, X_new_svd, xi_old, xi_new, eta_old)
            
            # Determine an initial step ti from the linearized problem
            t = self.compute_initial_guess(X_new_svd, Rw, eta_new)
            
            # Performing Armijo backtracking
            # Without it now 
            
            # For retraction
            m = 1
            step = (0.5**m)*t
            M_step = xi_new[0]*step
            Up_step = xi_new[1]*step
            Vp_step = xi_new[2]*step
            xi_new_step = [M_step, Up_step, Vp_step]
            
            # Compute Retraction
            X_cur_svd = self.compute_retraction(X_new_svd, xi_new_step)
            
            X_old_svd = copy.deepcopy(X_new_svd)
            X_new_svd = copy.deepcopy(X_cur_svd)